In [1]:
import numpy as np
import pandas as pd
import sys

from eppy import eppy
from eppy import hvacbuilder
from eppy import modeleditor
from eppy.modeleditor import IDF

In [2]:
# this function determines what idf version and connects the idd for eppy
def eppy_connection(idffile):
    fp = open(idffile)
    lines=fp.readlines()
    vers = ""
    vers = vers.join(lines[2:12])
    vers
    if vers.find('8.7') != -1:
        print('Using EnergyPlus version 8.7')
        iddfile = 'C:\\EnergyPlusV8-7-0\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    elif vers.find('8.8') != -1:
        print('Using EnergyPlus version 8.8')
        iddfile = 'C:\\EnergyPlusV8-8-0\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    elif vers.find('8.9') != -1:
        print('Using EnergyPlus version 8.9')
        iddfile = 'C:\\EnergyPlusV8-9-0\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    elif vers.find('9.0') != -1:
        print('Using EnergyPlus version 9.0.1')
        iddfile = 'C:\\EnergyPlusV9-0-1\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    else:
        print('IDF Type not found on line 3')

In [3]:
#define paths
commonSlab_path = 'C:\\Users\\scott\\github\\DOE_LRMF\\SEED_Models\\common_hp_slab_IECC_2012_V8.7.idf'
commonBsmt_path = 'C:\\Users\\scott\\github\\DOE_LRMF\\SEED_Models\\common_hp_heatedbsmtIECC_2012_V8.7.idf'
gardenSlab_path = 'C:\\Users\\scott\\github\\DOE_LRMF\\SEED_Models\\garden_hp_slab_IECC_2012_V8.7.idf'
gardenBsmt_path = 'C:\\Users\\scott\\github\\DOE_LRMF\\SEED_Models\\garden_hp_heatedbsmtIECC_2012_V8.7.idf'

#make connections
commonSlab = eppy_connection(commonSlab_path)
commonBsmt = eppy_connection(commonBsmt_path)
gardenSlab = eppy_connection(gardenSlab_path)
gardenBsmt = eppy_connection(gardenBsmt_path)

# create groups
idfs = [commonSlab, commonBsmt, gardenSlab, gardenBsmt]
common = [commonSlab, commonBsmt]
garden = [gardenSlab, gardenBsmt]
bsmt = [commonBsmt, gardenBsmt]
slab = [commonSlab, gardenSlab]

Using EnergyPlus version 8.7
Using EnergyPlus version 8.7
Using EnergyPlus version 8.7
Using EnergyPlus version 8.7


In [4]:
## LIST SURFACES TO MODIFY

commonSurfaces = ['Floor_unit1_FrontRow_BottomFloor', 'Floor_unit2_FrontRow_BottomFloor', 
            'Floor_unit3_FrontRow_BottomFloor', 'Floor_unit1_BackRow_BottomFloor', 
            'Floor_unit2_BackRow_BottomFloor', 'Floor_unit3_BackRow_BottomFloor', 
            'g Floor C']

gardenSurfaces = ['Floor_unit1_FrontRow_BottomFloor', 'Floor_unit2_FrontRow_BottomFloor', 
            'Floor_unit3_FrontRow_BottomFloor', 'Floor_unit1_BackRow_BottomFloor', 
            'Floor_unit2_BackRow_BottomFloor', 'Floor_unit3_BackRow_BottomFloor']

In [5]:
## CREATE NEW IDF OBJECTS

#common
commonBsmt.newidfobject('Material')
commonBsmt.idfobjects['Material'][-1].Name = 'exposed_floor_consol_layer'
commonBsmt.idfobjects['Material'][-1].Roughness = 'Rough'
commonBsmt.idfobjects['Material'][-1].Thickness = 0.23495
commonBsmt.idfobjects['Material'][-1].Conductivity = 0.0485233
commonBsmt.idfobjects['Material'][-1].Density = 55.074
commonBsmt.idfobjects['Material'][-1].Specific_Heat = 916.9311


commonBsmt.newidfobject('Construction')
commonBsmt.idfobjects['Construction'][-1].Name = 'Exposed Floor'
commonBsmt.idfobjects['Construction'][-1].Outside_Layer = 'exposed_floor_consol_layer'
commonBsmt.idfobjects['Construction'][-1].Layer_2 = 'Plywood_3/4in'
commonBsmt.idfobjects['Construction'][-1].Layer_3 = 'Carpet_n_pad'

#garden
gardenBsmt.newidfobject('Material')
gardenBsmt.idfobjects['Material'][-1].Name = 'exposed_floor_consol_layer'
gardenBsmt.idfobjects['Material'][-1].Roughness = 'Rough'
gardenBsmt.idfobjects['Material'][-1].Thickness = 0.23495
gardenBsmt.idfobjects['Material'][-1].Conductivity = 0.0485233
gardenBsmt.idfobjects['Material'][-1].Density = 55.074
gardenBsmt.idfobjects['Material'][-1].Specific_Heat = 916.9311


gardenBsmt.newidfobject('Construction')
gardenBsmt.idfobjects['Construction'][-1].Name = 'Exposed Floor'
gardenBsmt.idfobjects['Construction'][-1].Outside_Layer = 'exposed_floor_consol_layer'
gardenBsmt.idfobjects['Construction'][-1].Layer_2 = 'Plywood_3/4in'
gardenBsmt.idfobjects['Construction'][-1].Layer_3 = 'Carpet_n_pad'

In [6]:
## LOOP THROUGH SURFACES AND CHANGE CONSTRUCTIONS

# common basement
surfaces = commonBsmt.idfobjects['BuildingSurface:Detailed']
for i in range(0, len(commonSurfaces)):
    
    # grab surface from list of surfaces
    surface = [x for x in surfaces if x.Name == commonSurfaces[i]][0]
    surface.Construction_Name = 'Exposed Floor'

    #display changes
    print('Changed '+surface.Name+' construction to '+surface.Construction_Name)
    
# garden basement
surfaces = gardenBsmt.idfobjects['BuildingSurface:Detailed']
for i in range(0, len(gardenSurfaces)):
    
    # grab surfaces from list of surfaces
    surface = [x for x in surfaces if x.Name == gardenSurfaces[i]][0]
    surface.Construction_Name = 'Exposed Floor'
    
    #display changes
    print('Changed '+surface.Name+' construction to '+surface.Construction_Name)

Changed Floor_unit1_FrontRow_BottomFloor construction to Exposed Floor
Changed Floor_unit2_FrontRow_BottomFloor construction to Exposed Floor
Changed Floor_unit3_FrontRow_BottomFloor construction to Exposed Floor
Changed Floor_unit1_BackRow_BottomFloor construction to Exposed Floor
Changed Floor_unit2_BackRow_BottomFloor construction to Exposed Floor
Changed Floor_unit3_BackRow_BottomFloor construction to Exposed Floor
Changed g Floor C construction to Exposed Floor
Changed Floor_unit1_FrontRow_BottomFloor construction to Exposed Floor
Changed Floor_unit2_FrontRow_BottomFloor construction to Exposed Floor
Changed Floor_unit3_FrontRow_BottomFloor construction to Exposed Floor
Changed Floor_unit1_BackRow_BottomFloor construction to Exposed Floor
Changed Floor_unit2_BackRow_BottomFloor construction to Exposed Floor
Changed Floor_unit3_BackRow_BottomFloor construction to Exposed Floor


In [7]:
## save idfs
[x.save() for x in idfs]

[None, None, None, None]